In [1]:
import wandb
from tokenizers import Tokenizer
from model import GoePT
import json
import os
import cupy as cp
import torch
wandb.init(mode="disabled")

checkpoint_filename = "goe_sixthree.json"
with open(
    os.path.join("../../checkpoints", checkpoint_filename),
    mode="r",
    encoding="utf-8",
) as in_file:
    state_dict = json.load(in_file)
model_loaded = GoePT.from_state_dict(state_dict)



tokenizer: Tokenizer = Tokenizer.from_file("../tokenizers/goe_pt/goe_pt_tokenizer.json")
# tokenizer.enable_padding(length=256)




In [2]:
text = """Faust ging"""
tokenized = tokenizer.encode(text)
print(tokenized.ids)
padded_tokenized = cp.array(tokenized.ids)
tokenized = padded_tokenized.reshape((1, -1))
print(tokenized)

[1577, 1459]
[[1577 1459]]


In [5]:

logits, _ = model_loaded.forward(tokenized)


print(logits)
out = torch.topk(torch.tensor(logits),10,sorted=True)[1]
out = out.cpu().numpy().flatten()
print(out)
out = [tokenizer.decode((val,)) for val in out]
print(out)



ValueError: operands could not be broadcast together with shapes (1, 2, 1152) (256, 1152) (1, 2, 1152)